In [ ]:
import pandas as pd
from keras.callbacks import EarlyStopping
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ModelCheckpoint
from tabulate import tabulate
from src.datasets import camvid
from src import evaluate
from src import plot
from src import predict
from src import predict_video
from src.tiramisu import tiramisu
from src.wrappers import MonteCarlo
%matplotlib inline

In [ ]:
# the location to save fine training weights to
weights_file = 'models/Tiramisu103-CamVid11-fine.h5'

In [ ]:
# the size to crop images to for fine tune training
crop_size = (352, 480)

In [ ]:
camvid11 = camvid.CamVid(
    mapping=camvid.CamVid.load_mapping(),
    target_size=(360, 480),
    crop_size=crop_size,
    horizontal_flip=True,
    ignored_labels=['Void'],
)
generators = camvid11.generators()

In [ ]:
# get the next X, y training tuple
X, y = next(generators['train'])
# transform the onehot vector to an image
y = camvid11.unmap(y)
# plot the images
_ = plot.plot(X=X[0], y=y[0], order=['X', 'y'])

In [ ]:
# build the model for the image shape and number of labels
model = tiramisu.tiramisu((*crop_size, 3), camvid11.n, class_weights=camvid11.class_mask, mc_dropout=True)
model.load_weights(weights_file)

# Test Metrics

In [ ]:
# wrap the model with the test time Monte Carlo
model = MonteCarlo(model, 50, uncertainty='entropy')

In [ ]:
metrics = evaluate.evaluate(model, generators['test'], 233,
    mask=camvid11.class_mask, 
    code_map=camvid11.discrete_to_label_map,
)
metrics.to_csv('models/Tiramisu103-CamVid11-Epistemic.h5.csv')
metrics

In [ ]:
print(tabulate(metrics, tablefmt='pipe', headers=('Metric', 'Value')))

# Qualitative Results

## Train

In [ ]:
X, y, p, s = predict.predict_epistemic(model, generators['train'], camvid11)

In [ ]:
_ = plot.plot(X=X[0], y=y[0], y_pred=p[0], epistemic=s[0], order=['X', 'y', 'y_pred', 'epistemic'])

In [ ]:
_ = plot.plot(X=X[1], y=y[1], y_pred=p[1], epistemic=s[1], order=['X', 'y', 'y_pred', 'epistemic'])

In [ ]:
_ = plot.plot(X=X[2], y=y[2], y_pred=p[2], epistemic=s[2], order=['X', 'y', 'y_pred', 'epistemic'])

## Validation

In [ ]:
X, y, p, s = predict.predict_epistemic(model, generators['val'], camvid11)
_ = plot.plot(X=X[0], y=y[0], y_pred=p[0], epistemic=s[0], order=['X', 'y', 'y_pred', 'epistemic'])

In [ ]:
X, y, p, s = predict.predict_epistemic(model, generators['val'], camvid11)
_ = plot.plot(X=X[0], y=y[0], y_pred=p[0], epistemic=s[0], order=['X', 'y', 'y_pred', 'epistemic'])

In [ ]:
X, y, p, s = predict.predict_epistemic(model, generators['val'], camvid11)
_ = plot.plot(X=X[0], y=y[0], y_pred=p[0], epistemic=s[0], order=['X', 'y', 'y_pred', 'epistemic'])

In [ ]:
X, y, p, s = predict.predict_epistemic(model, generators['val'], camvid11)
_ = plot.plot(X=X[0], y=y[0], y_pred=p[0], epistemic=s[0], order=['X', 'y', 'y_pred', 'epistemic'])

## Test

In [ ]:
X, y, p, s = predict.predict_epistemic(model, generators['test'], camvid11)
fig = plot.plot(X=X[0], y=y[0], y_pred=p[0], epistemic=s[0], order=['X', 'y', 'y_pred', 'epistemic'])
fig.savefig('img/tiramisu-bayesian/epistemic/0.png', transparent=True, bbox_inches='tight')

In [ ]:
X, y, p, s = predict.predict_epistemic(model, generators['test'], camvid11)
fig = plot.plot(X=X[0], y=y[0], y_pred=p[0], epistemic=s[0], order=['X', 'y', 'y_pred', 'epistemic'])
fig.savefig('img/tiramisu-bayesian/epistemic/1.png', transparent=True, bbox_inches='tight')

In [ ]:
X, y, p, s = predict.predict_epistemic(model, generators['test'], camvid11)
fig = plot.plot(X=X[0], y=y[0], y_pred=p[0], epistemic=s[0], order=['X', 'y', 'y_pred', 'epistemic'])
fig.savefig('img/tiramisu-bayesian/epistemic/2.png', transparent=True, bbox_inches='tight')

In [ ]:
X, y, p, s = predict.predict_epistemic(model, generators['test'], camvid11)
fig = plot.plot(X=X[0], y=y[0], y_pred=p[0], epistemic=s[0], order=['X', 'y', 'y_pred', 'epistemic'])
fig.savefig('img/tiramisu-bayesian/epistemic/3.png', transparent=True, bbox_inches='tight')

# Video

In [ ]:
model.simulations = 7

In [ ]:
video_file = '0005VD.mp4'
video_path = camvid.videos.abs_path(video_file)
out_path = 'img/tiramisu-bayesian/epistemic/{}'.format(video_file)

In [ ]:
predict_video.predict_video(video_path, out_path, camvid11, model, predict.predict_epistemic)